In [17]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.keys import Keys
import time

In [45]:
driver = webdriver.Chrome('../../../../Files/chromedriver/chromedriver.exe')


In [167]:
#itsub url
base_url="https://www.youtube.com/channel/UCdUcjkyZtf-1WJyPPiETF1g/videos"
video_list = []

In [168]:
#video list
driver.get(base_url)

body = driver.find_element_by_tag_name('body')

#page 넘기기
num_page_down = 30
while num_page_down:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(5)
    num_page_down -= 1
    pass

html=driver.page_source
bsObject = BeautifulSoup(html,"html.parser")

#video가져오기
word="/watch?"
cnt=0
for link in bsObject.find_all('a'):
    try :
        if (link.get('href')[:len(word)] == word) & (cnt==0) :
            video_list.append(link.get('href'))
            cnt+=1
        else :
            cnt=0
    except TypeError :
        continue
    pass

In [178]:
#Data Frame
#video
video_columes =['Title','View','Likes']
video_data = pd.DataFrame(columns=video_columes)

#commnet
commnet_columes =['Title','Comment','Likes']
comment_data = pd.DataFrame(columns=commnet_columes)

In [179]:
#data crawling
youtube="https://www.youtube.com/"
for video in video_list :
    #video 탐색
    driver.get(youtube+video)
    body = driver.find_element_by_tag_name('body')
    num_page_down = 1#20
    while num_page_down:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        num_page_down -= 1
        pass
    
    video_html=driver.page_source
    video_bsObject = BeautifulSoup(video_html,"html.parser")
    #print(video_bsObject)
    
    #title
    title_find = video_bsObject.find_all(class_="ytp-title-link yt-uix-sessionlink ytp-title-fullerscreen-link")
    title=title_find[0].text.strip()#.encode('utf8')

  
    #hash tag
    hashtag_find = video_bsObject.find_all(class_="yt-simple-endpoint style-scope yt-formatted-string")
    cnt = 0
    tag_list=[]
    for word in hashtag_find :
        tag = word.text.strip()
        if (tag[0]=='#') :
            tag_list.append(tag)
            cnt+=1
        pass
    
    #view
    view_find = video_bsObject.find_all(class_="view-count style-scope yt-view-count-renderer")[0]
    temp_view = view_find.text.strip()
    view = temp_view[4:len(temp_view)-1].replace(",","").encode('utf8')

    #like
    like_find = video_bsObject.find_all(class_="hidden style-scope paper-tooltip")[1].text.strip()
    idx = like_find.find("/")-1
    video_like = int(like_find[:idx].replace(",",""))
    
    if not cnt ==0 :
        tag_list = tag_list[:int(cnt/2)]
    
    video_data=video_data.append(pd.DataFrame(data=[[title,int(view),video_like]],columns=video_columes))
   
#     break
    #page down으로 모든 commnet 불러오기
    
#     for 
#         #모든 commnet 저장
        
#         pass
    pass

KeyboardInterrupt: 

In [180]:
video_data

,Title,View,Likes
0,LG의 레전드? LG가 핸드폰으로 삼성을 찌바르던 시절. LG 초콜릿폰 다시 살펴보기!,314252.0,6372.0
0,구독자 100만 기념! 3세대 골드 버튼 언빡싱&감사한 마음으로 애플 에어팟 프로 ...,663304.0,68096.0
0,비싼게 좋은것 만은 아니다? 애플워치5 살 때 알루미늄vs스테인리스 선택법. 애플워...,306274.0,4009.0
0,미개봉 태블릿이 12만원? 할인하면 꼭! 사세요. 최고의 갓성비 아마존 Fire H...,541311.0,5879.0
0,분필? 중국당면인줄;; 에어팟보다 1/3 저렴한 샤오미 에어2 언빡싱! 뭐든지 만드...,331170.0,4325.0
0,우리 럭시... 다시 켜질까요? 갈기갈기 분해된 갤럭시 폴드 재조립 도전!,287710.0,4989.0
